In [2]:
import os, mlflow
from dotenv import load_dotenv

load_dotenv(override=True)  # Carga las variables del archivo .env
EXPERIMENT_NAME = "/Users/isabel.valladolid@iteso.mx/nyc-taxi-experiments"

mlflow.set_tracking_uri("databricks")
experiment = mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

In [3]:
import pickle
import pandas as pd
from sklearn.metrics import  root_mean_squared_error
from sklearn.feature_extraction import  DictVectorizer

In [4]:
def read_dataframe(filename):

    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)

    return df

In [5]:
df_train = read_dataframe('../data/green_tripdata_2025-01.parquet')
df_val = read_dataframe('../data/green_tripdata_2025-02.parquet')

Feature Engineering + One Hot Encoding

In [6]:
def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)

In [7]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dicts)

X_val = preprocess(df_val, dv)


target

In [8]:
target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

Definir los dataset como objetos de mlflow para poderlos trackear

In [9]:
training_dataset = mlflow.data.from_numpy(X_train.data, targets=y_train, name="green_tripdata_2025-01")
validation_dataset = mlflow.data.from_numpy(X_val.data, targets=y_val, name="green_tripdata_2025-02")

In [10]:
import math
import optuna
import pathlib
import xgboost as xgb
from optuna.samplers import TPESampler
from mlflow.models.signature import infer_signature

In [11]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

Función Objetivo

In [12]:
# ------------------------------------------------------------
# Definir la función objetivo para Optuna
#    - Recibe un `trial`, que se usa para proponer hiperparámetros.
#    - Entrena un modelo con esos hiperparámetros.
#    - Calcula la métrica de validación (RMSE) y la retorna (Optuna la minimizará).
#    - Abrimos un run anidado de MLflow para registrar cada trial.
# ------------------------------------------------------------
def objective(trial: optuna.trial.Trial):
    # Hiperparámetros MUESTREADOS por Optuna en CADA trial.
    # Nota: usamos log=True para emular rangos log-uniformes (similar a loguniform).
    params = {
        "max_depth": trial.suggest_int("max_depth", 4, 100),
        "learning_rate": trial.suggest_float("learning_rate", math.exp(-3), 1.0, log=True),
        "reg_alpha": trial.suggest_float("reg_alpha",   math.exp(-5), math.exp(-1), log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", math.exp(-6), math.exp(-1), log=True),
        "min_child_weight": trial.suggest_float("min_child_weight", math.exp(-1), math.exp(3), log=True),
        "objective": "reg:squarederror",  
        "seed": 42,                      
    }

    # Run anidado para dejar rastro de cada trial en MLflow
    with mlflow.start_run(nested=True):
        mlflow.set_tag("model_family", "xgboost")  # etiqueta informativa
        mlflow.log_params(params)                  # registra hiperparámetros del trial

        # Entrenamiento con early stopping en el conjunto de validación
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=100,
            evals=[(valid, "validation")],
            early_stopping_rounds=10,
        )

        # Predicción y métrica en validación
        y_pred = booster.predict(valid)
        rmse = root_mean_squared_error(y_val, y_pred)

        # Registrar la métrica principal
        mlflow.log_metric("rmse", rmse)

        # La "signature" describe la estructura esperada de entrada y salida del modelo:
        # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
        # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
        signature = infer_signature(X_val, y_pred)

        # Guardar el modelo del trial como artefacto en MLflow.
        mlflow.xgboost.log_model(
            booster,
            name="model",
            input_example=X_val[:5],
            signature=signature
        )

    # Optuna minimiza el valor retornado
    return rmse

Flujo de búsqueda

In [13]:
mlflow.xgboost.autolog(log_models=False)

# ------------------------------------------------------------
# Crear el estudio de Optuna
#    - Usamos TPE (Tree-structured Parzen Estimator) como sampler.
#    - direction="minimize" porque queremos minimizar el RMSE.
# ------------------------------------------------------------
sampler = TPESampler(seed=42)
study = optuna.create_study(direction="minimize", sampler=sampler)

# ------------------------------------------------------------
# Ejecutar la optimización (n_trials = número de intentos)
#    - Cada trial ejecuta la función objetivo con un set distinto de hiperparámetros.
#    - Abrimos un run "padre" para agrupar toda la búsqueda.
# ------------------------------------------------------------
with mlflow.start_run(run_name="XGBoost Hyperparameter Optimization (Optuna)", nested=True):
    study.optimize(objective, n_trials=3)

    # --------------------------------------------------------
    # Recuperar y registrar los mejores hiperparámetros
    # --------------------------------------------------------
    best_params = study.best_params
    # Asegurar tipos/campos fijos (por claridad y consistencia)
    best_params["max_depth"] = int(best_params["max_depth"])
    best_params["seed"] = 42
    best_params["objective"] = "reg:squarederror"

    mlflow.log_params(best_params)

    # Etiquetas del run "padre" (metadatos del experimento)
    mlflow.set_tags({
        "project": "NYC Taxi Time Prediction Project",
        "optimizer_engine": "optuna",
        "model_family": "xgboost",
        "feature_set_version": 1,
    })

    # --------------------------------------------------------
    # 7) Entrenar un modelo FINAL con los mejores hiperparámetros
    #    (normalmente se haría sobre train+val o con CV; aquí mantenemos el patrón original)
    # --------------------------------------------------------
    booster = xgb.train(
        params=best_params,
        dtrain=train,
        num_boost_round=100,
        evals=[(valid, "validation")],
        early_stopping_rounds=10,
    )

    # Evaluar y registrar la métrica final en validación
    y_pred = booster.predict(valid)
    rmse = root_mean_squared_error(y_val, y_pred)
    mlflow.log_metric("rmse", rmse)

    # --------------------------------------------------------
    # 8) Guardar artefactos adicionales (p. ej. el preprocesador)
    # --------------------------------------------------------
    pathlib.Path("preprocessor").mkdir(exist_ok=True)
    with open("preprocessor/preprocessor.b", "wb") as f_out:
        pickle.dump(dv, f_out)

    mlflow.log_artifact("preprocessor/preprocessor.b", artifact_path="preprocessor")

    # La "signature" describe la estructura esperada de entrada y salida del modelo:
    # incluye los nombres, tipos y forma (shape) de las variables de entrada y el tipo de salida.
    # MLflow la usa para validar datos en inferencia y documentar el modelo en el Model Registry.
    # Si X_val es la matriz dispersa (scipy.sparse) salida de DictVectorizer:
    feature_names = dv.get_feature_names_out()
    input_example = pd.DataFrame(X_val[:5].toarray(), columns=feature_names)

    # Para que las longitudes coincidan, usa el mismo slice en y_pred
    signature = infer_signature(input_example, y_val[:5])

    # Guardar el modelo del trial como artefacto en MLflow.
    mlflow.xgboost.log_model(
        booster,
        name="model",
        input_example=input_example,
        signature=signature
    )

[I 2025-10-21 21:23:57,563] A new study created in memory with name: no-name-388c8a83-56d3-4e32-ab15-abea90abe113


[0]	validation-rmse:5.98960
[1]	validation-rmse:5.92413
[2]	validation-rmse:5.92287
[3]	validation-rmse:5.92324
[4]	validation-rmse:5.92483
[5]	validation-rmse:5.92489
[6]	validation-rmse:5.92498
[7]	validation-rmse:5.92343
[8]	validation-rmse:5.92347
[9]	validation-rmse:5.92467
[10]	validation-rmse:5.92456
[11]	validation-rmse:5.92460


/Users/isabelvalladolid/Documents/ProyectoCienciaDatos/nyc-taxi-predictions-2025/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [21:24:06] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/21 21:24:08 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
/Users/isabelvalladolid/Documents/ProyectoCienciaDatos/nyc-taxi-predictions-2025/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:321: UserWarning: [21:24:08] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
2025/10/21 21:24:08 INFO mlflow.models.model: Found the following environment variabl

🏃 View run useful-donkey-563 at: https://dbc-00d2933e-633b.cloud.databricks.com/ml/experiments/1858882611825285/runs/802c5c4ea4dd49d489c2e06da352a25a
🧪 View experiment at: https://dbc-00d2933e-633b.cloud.databricks.com/ml/experiments/1858882611825285
[0]	validation-rmse:8.79510
[1]	validation-rmse:8.50586
[2]	validation-rmse:8.24215
[3]	validation-rmse:8.00493
[4]	validation-rmse:7.78830
[5]	validation-rmse:7.59575
[6]	validation-rmse:7.41983
[7]	validation-rmse:7.26129
[8]	validation-rmse:7.11270
[9]	validation-rmse:6.98583
[10]	validation-rmse:6.87056
[11]	validation-rmse:6.76533
[12]	validation-rmse:6.67029
[13]	validation-rmse:6.58378
[14]	validation-rmse:6.50773
[15]	validation-rmse:6.43768
[16]	validation-rmse:6.37633
[17]	validation-rmse:6.32116
[18]	validation-rmse:6.27142
[19]	validation-rmse:6.22765
[20]	validation-rmse:6.18963
[21]	validation-rmse:6.15802
[22]	validation-rmse:6.12794
[23]	validation-rmse:6.10295
[24]	validation-rmse:6.07940
[25]	validation-rmse:6.05863
[26]	

/Users/isabelvalladolid/Documents/ProyectoCienciaDatos/nyc-taxi-predictions-2025/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [21:24:16] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/21 21:24:17 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
/Users/isabelvalladolid/Documents/ProyectoCienciaDatos/nyc-taxi-predictions-2025/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:321: UserWarning: [21:24:17] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-21 21:24:19,281] Trial 1 finished with value: 5.902611653559419 and parame

🏃 View run blushing-auk-105 at: https://dbc-00d2933e-633b.cloud.databricks.com/ml/experiments/1858882611825285/runs/86b949cd53b04786ad2cea75e8604255
🧪 View experiment at: https://dbc-00d2933e-633b.cloud.databricks.com/ml/experiments/1858882611825285
[0]	validation-rmse:5.90396
[1]	validation-rmse:5.88901
[2]	validation-rmse:5.88875
[3]	validation-rmse:5.88908
[4]	validation-rmse:5.88973
[5]	validation-rmse:5.88241
[6]	validation-rmse:5.88348
[7]	validation-rmse:5.88292
[8]	validation-rmse:5.87483
[9]	validation-rmse:5.87457
[10]	validation-rmse:5.87441
[11]	validation-rmse:5.87453
[12]	validation-rmse:5.87449
[13]	validation-rmse:5.87444
[14]	validation-rmse:5.87358
[15]	validation-rmse:5.87331
[16]	validation-rmse:5.87310
[17]	validation-rmse:5.87302
[18]	validation-rmse:5.87068
[19]	validation-rmse:5.87074
[20]	validation-rmse:5.87061
[21]	validation-rmse:5.87062
[22]	validation-rmse:5.87063
[23]	validation-rmse:5.87068
[24]	validation-rmse:5.87073
[25]	validation-rmse:5.87077
[26]	v

/Users/isabelvalladolid/Documents/ProyectoCienciaDatos/nyc-taxi-predictions-2025/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [21:24:22] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/21 21:24:24 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
/Users/isabelvalladolid/Documents/ProyectoCienciaDatos/nyc-taxi-predictions-2025/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:321: UserWarning: [21:24:24] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)
[I 2025-10-21 21:24:25,274] Trial 2 finished with value: 5.873624736518469 and parame

🏃 View run delightful-sloth-212 at: https://dbc-00d2933e-633b.cloud.databricks.com/ml/experiments/1858882611825285/runs/27f53d3db1a2426b8a3597c9a1dcb370
🧪 View experiment at: https://dbc-00d2933e-633b.cloud.databricks.com/ml/experiments/1858882611825285
[0]	validation-rmse:5.90396
[1]	validation-rmse:5.88901
[2]	validation-rmse:5.88875
[3]	validation-rmse:5.88908
[4]	validation-rmse:5.88973
[5]	validation-rmse:5.88241
[6]	validation-rmse:5.88348
[7]	validation-rmse:5.88292
[8]	validation-rmse:5.87483
[9]	validation-rmse:5.87457
[10]	validation-rmse:5.87441
[11]	validation-rmse:5.87453
[12]	validation-rmse:5.87449
[13]	validation-rmse:5.87444
[14]	validation-rmse:5.87358
[15]	validation-rmse:5.87331
[16]	validation-rmse:5.87310
[17]	validation-rmse:5.87302
[18]	validation-rmse:5.87068
[19]	validation-rmse:5.87074
[20]	validation-rmse:5.87061
[21]	validation-rmse:5.87062
[22]	validation-rmse:5.87063
[23]	validation-rmse:5.87068
[24]	validation-rmse:5.87073
[25]	validation-rmse:5.87077
[2

/Users/isabelvalladolid/Documents/ProyectoCienciaDatos/nyc-taxi-predictions-2025/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:169: UserWarning: [21:24:29] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1575: Saving model in the UBJSON format as default.  You can use a file extension: `json` or `ubj` to choose between formats.
  xgb_model.save_model(model_data_path)
2025/10/21 21:24:30 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.
/Users/isabelvalladolid/Documents/ProyectoCienciaDatos/nyc-taxi-predictions-2025/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:321: UserWarning: [21:24:30] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


🏃 View run XGBoost Hyperparameter Optimization (Optuna) at: https://dbc-00d2933e-633b.cloud.databricks.com/ml/experiments/1858882611825285/runs/3ca5362d71134e7baa7d3fd3859af185
🧪 View experiment at: https://dbc-00d2933e-633b.cloud.databricks.com/ml/experiments/1858882611825285


Registrar modelo en Model Registry

In [14]:
bmodel_name = "workspace.default.nyc-taxi-model"

In [15]:
run_id = input("Ingrese el run_id")
run_uri = f"runs:/{run_id}/model"

result = mlflow.register_model(
    model_uri=run_uri,
    name="workspace.default.nyc-taxi-model"
)

Successfully registered model 'workspace.default.nyc-taxi-model'.
2025/10/21 21:25:54 WARNING mlflow.tracking._model_registry.fluent: Run with id 3ca5362d71134e7baa7d3fd3859af185 has no artifacts at artifact path 'model', registering model based on models:/m-31c1eee3ee0b46b28b71e7565552b2aa instead


Uploading artifacts:   0%|          | 0/8 [00:00<?, ?it/s]

Created version '1' of model 'workspace.default.nyc-taxi-model'.


In [16]:
runs = mlflow.search_runs(
    experiment_names=[EXPERIMENT_NAME],
    order_by=["metrics.rmse ASC"],
    output_format="list"
)

# Obtener el mejor run
if len(runs) > 0:
    best_run = runs[0]
    print("🏆 Champion Run encontrado:")
    print(f"Run ID: {best_run.info.run_id}")
    print(f"RMSE: {best_run.data.metrics['rmse']}")
    print(f"Params: {best_run.data.params}")
else:
    print("⚠️ No se encontraron runs con métrica RMSE.")

🏆 Champion Run encontrado:
Run ID: 27f53d3db1a2426b8a3597c9a1dcb370
RMSE: 5.873624736518469
Params: {'early_stopping_rounds': '10', 'custom_metric': 'None', 'verbose_eval': 'True', 'seed': '42', 'reg_lambda': '0.007166739666045858', 'reg_alpha': '0.18820387978911576', 'objective': 'reg:squarederror', 'num_boost_round': '100', 'min_child_weight': '0.7613210498541186', 'maximize': 'None', 'max_depth': '5', 'learning_rate': '0.9136840519292246'}


In [18]:
run_id = best_run.info.run_id

In [20]:
result = mlflow.register_model(
    model_uri=f"runs:/{best_run.info.run_id}/model",
    name=bmodel_name
)

Registered model 'workspace.default.nyc-taxi-model' already exists. Creating a new version of this model...
2025/10/21 21:27:39 WARNING mlflow.tracking._model_registry.fluent: Run with id 27f53d3db1a2426b8a3597c9a1dcb370 has no artifacts at artifact path 'model', registering model based on models:/m-02e9bf302bde4370aaf02dcedffc0e19 instead


Uploading artifacts:   0%|          | 0/8 [00:00<?, ?it/s]

Created version '2' of model 'workspace.default.nyc-taxi-model'.


Asignar alias

In [22]:
from mlflow import MlflowClient

client = MlflowClient()

In [24]:
model_version = result.version
new_alias = "Champion"

client.set_registered_model_alias(
    name=bmodel_name,
    alias=new_alias,
    version=result.version
)

In [26]:
from datetime import datetime

date = datetime.today()

client.update_model_version(
    name=bmodel_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_alias} on {date}",
)

<ModelVersion: aliases=[], creation_timestamp=1761103661945, current_stage=None, deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='The model version 2 was transitioned to Champion on 2025-10-21 21:28:17.673384', last_updated_timestamp=1761103698029, metrics=[<Metric: dataset_digest='', dataset_name='', key='best_iteration', model_id='m-02e9bf302bde4370aaf02dcedffc0e19', run_id='27f53d3db1a2426b8a3597c9a1dcb370', step=0, timestamp=1761103460136, value=20.0>,
 <Metric: dataset_digest='', dataset_name='', key='rmse', model_id='m-02e9bf302bde4370aaf02dcedffc0e19', run_id='27f53d3db1a2426b8a3597c9a1dcb370', step=0, timestamp=1761103461771, value=5.873624736518469>,
 <Metric: dataset_digest='', dataset_name='', key='stopped_iteration', model_id='m-02e9bf302bde4370aaf02dcedffc0e19', run_id='27f53d3db1a2426b8a3597c9a1dcb370', s

Obteniendo modelos del Moldel Registry

In [28]:
	
import mlflow.pyfunc

model_version_uri = f"models:/{bmodel_name}@Champion"

champion_version = mlflow.pyfunc.load_model(model_version_uri)
champion_version.predict(X_val)

/Users/isabelvalladolid/Documents/ProyectoCienciaDatos/nyc-taxi-predictions-2025/.venv/lib/python3.11/site-packages/mlflow/xgboost/__init__.py:321: UserWarning: [21:28:42] WARNING: /Users/runner/work/xgboost/xgboost/src/c_api/c_api.cc:1511: Unknown file format: `xgb`. Using UBJSON (`ubj`) as a guess.
  model.load_model(xgb_model_path)


array([ 4.7416825, 24.05205  , 29.134676 , ..., 21.56244  , 13.990461 ,
       24.05205  ], shape=(44218,), dtype=float32)